### Predicting submodel outputs from description

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
nlp = spacy.load("en_core_web_lg")

In [3]:
data_train = pd.read_csv('cve_train.csv')
data_test = pd.read_csv('cve_test.csv')

# X - input: descriptions
# y - output: attack vector

data = pd.concat([data_train, data_test], ignore_index=True)
X = data['description']
y = data['attack_vector']

print(X[0])


A remote code exection vulnerability was identified in HPE Intelligent Management Center (IMC) PLAT earlier than version 7.3 E0506P09.


In [14]:
#Text cleaning (because tokenizer acts weirdly with numbers)
import re
def filter_websites(text):
    pattern = r'(http\:\/\/|https\:\/\/)?([a-z0-9][a-z0-9\-]*\.)+[a-z][a-z\-]*'
    text = re.sub(pattern, '', text)
    return text
def remove_version_numbers(text):
    pattern = r'?:(\d+)\.)?(?:(\d+)\.)?(?:(\d+)\.\d+'
    text = re.sub(pattern, '', text)
    return text

def clean_text(text):
    text = text.lower()
    text = filter_websites(text)
    #text = remove_version_numbers(text)
    return text

data['cleaned_description'] = data.description.apply(clean_text)
X = data['cleaned_description']

print(X[0])

a remote code exection vulnerability was identified in hpe intelligent management center (imc) plat earlier than version 7.3 e0506p09.


In [15]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [16]:
print(X[1])
print('Tokenized: ', tokenizer.tokenize(X[1]))

an information disclosure vulnerability in the android media framework (n/a). product: android. versions: 7.0, 7.1.1, 7.1.2, 8.0. android id: a-63873837.
Tokenized:  ['an', 'information', 'disclosure', 'vulnerability', 'in', 'the', 'android', 'media', 'framework', '(', 'n', '/', 'a', ')', '.', 'product', ':', 'android', '.', 'versions', ':', '7', '.', '0', ',', '7', '.', '1', '.', '1', ',', '7', '.', '1', '.', '2', ',', '8', '.', '0', '.', 'android', 'id', ':', 'a', '-', '63', '##8', '##7', '##38', '##37', '.']


In [21]:
#Turning string labels into numerical data
# 0 - network
# 1 - adjacent network
# 2 - local
# 3 - physical

data['attack_vector'] = data['attack_vector'].replace(['NETWORK', 'ADJACENT_NETWORK', 'LOCAL', 'PHYSICAL'], [0, 1, 2, 3])
y = data['attack_vector']

In [22]:
#finding the length of the longest description
max_len = 0
counter = 0

X_new = []
y_new = []

for description in range(len(X)):
    if len(X[description]) <= 512:
        X_new.append(X[description])
        y_new.append(y[description])

        #tokenize the text and add CLS and SEP tokens

        input_ids = tokenizer.encode(X[description], add_special_tokens=True)

        # Update the maximum description length
        max_len = max(max_len, len(input_ids))

print(len(X))
print(len(X_new))
print('Max description length: ', max_len)

77020
67295
Max description length:  291


In [23]:
# Tokenize all descriptions and map the tokens to their word IDs
import torch
input_ids = []
attention_masks = []

for description in X_new:
    encoded_dict = tokenizer.encode_plus(
                                description,
                                add_special_tokens = True,
                                max_length = 64,
                                pad_to_max_length = True,
                                return_attention_mask = True,
                                return_tensors = 'pt'    
                            )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

#convert the lists into tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(y_new)

print('Original: ', X_new[1])
print('Token IDs: ', input_ids[1])

C:\Users\Lenovo\miniconda3\lib\site-packages\transformers\tokenization_utils_base.py:2221: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  an information disclosure vulnerability in the android media framework (n/a). product: android. versions: 7.0, 7.1.1, 7.1.2, 8.0. android id: a-63873837.
Token IDs:  tensor([  101,  2019,  2592, 19380, 18130,  1999,  1996, 11924,  2865,  7705,
         1006,  1050,  1013,  1037,  1007,  1012,  4031,  1024, 11924,  1012,
         4617,  1024,  1021,  1012,  1014,  1010,  1021,  1012,  1015,  1012,
         1015,  1010,  1021,  1012,  1015,  1012,  1016,  1010,  1022,  1012,
         1014,  1012, 11924,  8909,  1024,  1037,  1011,  6191,  2620,  2581,
        22025, 24434,  1012,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


In [24]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, torch.Tensor(y_new))

# Create a 80-20 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

53,836 training samples
13,459 validation samples


In [25]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [27]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [30]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 4, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
model.to(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [31]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [32]:
from transformers import get_linear_schedule_with_warmup

epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [33]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [34]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [35]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0
    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()
    print("Treniram!")
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        print (b_input_ids.size())
        print (b_input_mask.size())
        print (b_labels.size())
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        loss, logits = model(b_input_ids.squeeze(), 
                       token_type_ids=None, 
                       attention_mask=b_input_mask.squeeze(), 
                       labels=b_labels.squeeze(),
                       return_dict=True)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        print("Backwardiram!")
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        print("Optimiziram!")
        optimizer.step()
        # Update the learning rate.
        print("Updateiram!")
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        


======== Epoch 1 / 2 ========
Training...
Treniram!
torch.Size([32, 64])
torch.Size([32, 64])
torch.Size([32])


ValueError: Target size (torch.Size([32])) must be the same as input size (torch.Size([32, 4]))